# BROCHURE BUILDER

In [ ]:
# pip install requests dotenv beautifulSoup4 openai

  Using cached openai-1.64.0-py3-none-any.whl.metadata (27 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp310-cp310-win_amd64.whl.metadata (5.3 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp310-cp310-win_amd64.whl.metadata (6.7 kB)
Using cached openai-1.64.0-py3-none-any.whl (472 kB)
Using cached anyio-4.8.0-py3-none-any.whl (96 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using c

In [6]:
import os
import requests
from openai import OpenAI
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [30]:
# Initialize the constants

load_dotenv(override=True)
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

if OPENAI_KEY and OPENAI_KEY.startswith("sk-proj-") and len(OPENAI_KEY) > 10:
    print("The api key looks good")
else:
    print("There might be a problem with the api key")

MODEL = 'gpt-4o-mini'
openai = OpenAI()



The api key looks good


In [20]:
# Class which represents webpage
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body():
            for irrelevant in soup.body(['script', 'img', 'input', 'style']):
                irrelevant.decompose()
        else :   
            self.text = ''
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

        def get_contents(self):
            return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        

# website = Website("https://www.cypherock.com/?srsltid=AfmBOooDbHMbrl25kt6F4EwMY8zR_Oit2VuZr-GODuBv5KasLaBdqUn9")
# website.links
        

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

# print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
                Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

# print(get_links_user_prompt(website))

In [ ]:
def get_links(website):
    website = Website("https://www.cypherock.com/?srsltid=AfmBOooDbHMbrl25kt6F4EwMY8zR_Oit2VuZr-GODuBv5KasLaBdqUn9")
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            { 'role':'system', 'content':link_system_prompt},
            {'role':'user', 'content':get_links_user_prompt(website)}
        ],
        response_format={'type':'json_object'}
    )

    result = response.choices[0].message.content
    return json.loads(result)

# get_links(website)

{'links': [{'type': 'about page', 'url': 'https://www.cypherock.com/about'},
  {'type': 'product page',
   'url': 'https://www.cypherock.com/product/cypherock-x1'},
  {'type': 'how it works page',
   'url': 'https://www.cypherock.com/how-it-works'},
  {'type': 'features page', 'url': 'https://www.cypherock.com/features'},
  {'type': 'careers page', 'url': 'https://angel.co/company/cypherock-wallet'},
  {'type': 'faq page', 'url': 'https://www.cypherock.com/faq'},
  {'type': 'company news',
   'url': 'https://www.entrepreneur.com/en-in/news-and-trends/cypherock-raises-1million-to-build-mpc-based-hardware/440209'}]}